# Colour classifying neural network classification regions
This is an interactive Jupyter Notebook for showing the colour classification distributions of my colour classifying neural network.

Importing all the things that we need, also setting the filename for the weights file.

In [1]:
from main import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

FILENAME = "weights.npz"

Importing the weights from the file.

In [2]:
hl_weights, ol_weights = import_weights(FILENAME)

Creating an interactive window to show a 2D representation of two of the RGB values changing as one is held constant, classifying those colours, and drawing separation lines between different colours.

In [16]:
@interact(slider=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.5), 
          slider_colour=widgets.RadioButtons(options=['Red', 'Green', 'Blue'],description='Select colour to change with slider:'))
def update_canvas(slider, slider_colour):
    selected_colour = slider_colour
    # Create a canvas with specified dimensions
    canvas_width = 100
    canvas_height = 100
    classification_step_size = 10
    half_step_size = int(classification_step_size/2)

    # Create a canvas with the gradient
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    for y in range(canvas_height):
        for x in range(canvas_width):
            # Interpolate the color at each pixel
            colour1 = x/canvas_width*255
            colour2 = y/canvas_height*255
            colour3 = slider*255
            
            if selected_colour == "Blue":
                canvas[y, x] = [colour1, colour2, colour3]
            elif selected_colour == "Green":
                canvas[y, x] = [colour1, colour3, colour2]
            elif selected_colour == "Red":
                canvas[y, x] = [colour3, colour1, colour2]
                
    classifications = []
    for i in range(0, canvas_height):
        current_row = []
        for j in range(0, canvas_width):
            current_colour = [c/255 for c in canvas[i, j]]
            current_row.append(arr_to_colour_name(forward_pass(np.array(current_colour), hl_weights, ol_weights)))
        
        classifications.append(current_row)
        
    for i in range(1, canvas_height-1):
        for j in range(1, canvas_width-1):
            same = True
            for di in range(-1, 2, 1):
                for dj in range(-1, 2, 1):
                    if classifications[j][i] != classifications[j+dj][i+di]:
                        same = False
                        break
                if not same:
                    break
            if not same:
                canvas[i, j] = [0, 0, 0]

    # Display the canvas with the color gradient
    plt.imshow(canvas)
    plt.axis('off')  # Turn off axis labels and ticks
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='slider', max=1.0), RadioButtons(description='Select …